<a href="https://colab.research.google.com/github/usnhee/TIL-/blob/master/ke_T5_QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Runtime -> Change Runtime -> GPU

In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.3 MB/s eta 0:14:24tcmalloc: large alloc 1147494400 bytes == 0x55b1de1dc000 @  0x7fe390ee4615 0x55b1a5a8b17c 0x55b1a5b6b47a 0x55b1a5a8df9d 0x55b1a5b7fd4d 0x55b1a5b01ec8 0x55b1a5afca2e 0x55b1a5a8f88a 0x55b1a5b01d30 0x55b1a5afca2e 0x55b1a5a8f88a 0x55b1a5afe719 0x55b1a5b80b76 0x55b1a5afdd95 0x55b1a5b80b76 0x55b1a5afdd95 0x55b1a5b80b76 0x55b1a5afdd95 0x55b1a5a8fce9 0x55b1a5ad3579 0x55b1a5a8e902 0x55b1a5b01c4d 0x55b1a5afca2e 0x55b1a5a8f88a 0x55b1a5afe719 0x55b1a5afca2e 0x55b1a5a8f88a 0x55b1a5afd8f6 0x55b1a5a8f7aa 0x55b1a5afdb4f 0x55b1a5afca2e
     |█████████████████               | 1055.7 MB 61.6 MB/s eta 0:00:16tcmalloc: large alloc 1434370048 bytes == 0x55b222832000 @  0x7fe390ee4615 0x55b1a5a8b17c 0x55b1a5b6b47a 0x55b1a5a8df9d 0x55b1a5b7fd4d 0x55b1a5b01ec8 0x55b1a5afca2e 0x55b1a5a8f88a 0x55b1a5b01d30 0x55b1a5afca2e 0x55b1a5a8f88a 0x55b1a5afe719 0x55b1a5b80b76 0x55b1a5afd

In [ ]:
!pip install --quiet "torchvision" "torchmetrics>=0.6" "pytorch-lightning>=1.4" "ipython[notebook]" "torch>=1.6, <1.9"

In [ ]:
!python -V

Python 3.7.13


In [ ]:
import torch

torch.__version__

In [ ]:
import pytorch_lightning as pl
print(pl.__version__)

1.6.1


In [ ]:
# check for the GPU provided in the runtime
!nvidia-smi
# using quiet method for controlling the log
# for suppressing the colored errors and warning in the terminal
!pip install --quiet transformers==4.1.1
# pytorch lightning for smoother model training and data loading
!pip install --quiet https://github.com/PyTorchLightning/pytorch-lightning/releases/download/1.2.6/pytorch-lightning-1.2.6.tar.gz 
# using HuggingFace tokenizers
!pip install --quiet tokenizers==0.9.4
# Google's sentencepiece
!pip install --quiet sentencepiece==0.1.94
# mostly pl is used while doing complex model training
import pytorch_lightning as pl
print(pl.__version__)
# argparse makes it easier to write user friendly command line interfaces
import argparse
# package for faster file name matching
import glob
# makiing directories for data 
import os
# reading json files as the data is present in json files
import json
# time module for calculating the model runtime
import time
# Allows writing status messages to a file
import logging
# generate random float numbers uniformly
import random
# regex module for text 
import re
# module provides various functions which work on 
# iterators too produce complex iterators
from itertools import chain
from string import punctuation
# pandas for data manipulation
import pandas as pd
# numpy for array operations
import numpy as np
# PyTorch
import torch
# provides various classes representing file system paths
# with appropriate semantics
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
# splitting the data 
from sklearn.model_selection import train_test_split
# ANSII color formatting for ouput in terminal
from termcolor import colored
# wrapping paragraphs into string
import textwrap
# model checkpoints in pretrained model
from pytorch_lightning.callbacks import ModelCheckpoint
'''
optimizer - AdamW
T5 Conditional Generator in which we'll give conditions
T5 tokenizer because it is fast
training the model without a learning rate
'''
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
# Seeds all the processes including numpy torch and other imported modules.
pl.seed_everything(0)
# check the version provided by Lightning
import pytorch_lightning as pl
print(pl.__version__)

In [ ]:
# RuntimeError: CUDA out of memory. Tried to allocate 134.00 MiB (GPU 0; 15.90 GiB total capacity; 14.35 GiB already allocated; 29.75 MiB free; 14.76 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation. See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
# 이런 에러가 발생하는 이유는 batch size가 너무 크거나, 코드 상에서 메모리 누수가 발생했기 때문이라고 한다.
# 출처 : https://beausty23.tistory.com/145

import gc
gc.collect()
torch.cuda.empty_cache()

# Downloading the Dataset

In [ ]:
import os
import json
import numpy as np
from tqdm.notebook import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/6조_파이널PJT/data/ko1.0_add_ko2.0.csv')

# Function to Create Pandas Dataframes of Questions and Answers
This function will help us read the data from the folder containing multiple JSON files and read them to a dataframe to run manipulation on it and proceed further. 

In [ ]:
# Dropping all the rows with repeated context and questions pairs.
# df = df.drop_duplicates(subset=["context"]).reset_index(drop=True)
df.shape
len(df.question.unique())
len(df.context.unique())
sample_question = df.iloc[243]
sample_question
# Using textcolor to visualize the answer within the context
def color_answer(question):
  answer_start, answer_end = question["answer_start"],question["answer_end"]
  context = question['context']
  return  colored(context[:answer_start], "white") + \
    colored(context[answer_start:answer_end + 1], "green") + \
    colored(context[answer_end+1:], "white")
print(sample_question['question'])
print()
print("Answer: ")
for wrap in textwrap.wrap(color_answer(sample_question), width = 100):
  print(wrap)

악티늄족을 핵 연료로 이용할 때 사용해야 하는 방사능 차단 물질은 어떤 종류인가?

Answer: 
악티늄족을 핵 연료로 이용할 때는, 방사능원을 차단하거나 방사성 물질을 막을 수 있는 고급 광물(예로 들자면 큰 잠재적 이득이 있는 자체
발광 결정 등)을 사용해야 한다. 그러나, 악티늄족의 강한 방사능독으로 인해 자연에서의 이동과 악티늄족에 깊은 우려가 있기도 하다. 혼합산화물핵연료(MOX)에서 화학적으로 불안정한
형태를 가진 악티늄족의 사용과 방사능 원천을 차단하는 것으로는 현대 안전 기준에는 적절하지 않다. 현재 안정되고 내구성이 좋은 악티늄족과 관련된 물질을 제작하는 것에 도전하고
있다. 이것은 안전한 창고에 들어있어야 하고, 사용하고 나서 최종 처분할 수 있어야 한다. 악티늄족 고체 용액의 적용의 열쇠는 바로 내구성이 좋은 결정체의 상이다.


# Tokenization
In the following cells, we have instantiated the model and called its tokenizer. T5 tokenizer is pretty fast as compared to other BERT type tokenizers. We will run a sample of this on the text given below and do the decoding.

In [ ]:
# using the base T5 model having 222M params
MODEL_NAME ='KETI-AIR/ke-t5-base' 
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
sample_encoding = tokenizer('is the glass half empty or half full?', 'It depends on the initial state of the glass. If the glass starts out empty and liquid is added until it is half full, it is half full. If the glass starts out full and liquid is removed until it is half empty, it is half empty.')
sample_encoding.keys()
print(sample_encoding["input_ids"])
print(sample_encoding["attention_mask"])
print(len(sample_encoding['input_ids']), len(sample_encoding['attention_mask']))
# Checking the decoding of the input ids
preds = [
        tokenizer.decode(input_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for input_id in sample_encoding['input_ids']
]
preds= " ".join(preds)
for wrap in textwrap.wrap(preds, width = 80):
  print(wrap)
encoding = tokenizer(
    sample_question['question'],
    sample_question['context'],
    max_length=396,
    padding='max_length',
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)
encoding.keys()
tokenizer.special_tokens_map
tokenizer.eos_token, tokenizer.eos_token_id
tokenizer.decode(encoding['input_ids'].squeeze()) 

[43, 5, 12857, 1648, 19876, 117, 1648, 1479, 85, 1, 223, 26000, 46, 5, 9113, 604, 14, 5, 12857, 3, 799, 5, 12857, 9162, 176, 19876, 13, 24483, 43, 1586, 1429, 60, 43, 1648, 1479, 4, 60, 43, 1648, 1479, 3, 799, 5, 12857, 9162, 176, 1479, 13, 24483, 43, 11514, 1429, 60, 43, 1648, 19876, 4, 60, 43, 1648, 19876, 3, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
63 63
is the glass half empty or half full ? </s> It depends on the initial state of
the glass . If the glass starts out empty and liquid is added until it is half
full , it is half full . If the glass starts out full and liquid is removed
until it is half empty , it is half empty . </s>


'악티늄족을 핵 연료로 이용할 때 사용해야 하는 방사능 차단 물질은 어떤 종류인가?</s> 악티늄족을 핵 연료로 이용할 때는, 방사능원을 차단하거나 방사성 물질을 막을 수 있는 고급 광물(예로 들자면 큰 잠재적 이득이 있는 자체 발광 결정 등)을 사용해야 한다. 그러나, 악티늄족의 강한 방사능독으로 인해 자연에서의 이동과 악티늄족에 깊은 우려가 있기도 하다. 혼합산화물핵연료(MOX)에서 화학적으로 불안정한 형태를 가진 악티늄족의 사용과 방사능 원천을 차단하는 것으로는 현대 안전 기준에는 적절하지 않다. 현재 안정되고 내구성이 좋은 악티늄족과 관련된 물질을 제작하는 것에 도전하고 있다. 이것은 안전한 창고에 들어있어야 하고, 사용하고 나서 최종 처분할 수 있어야 한다. 악티늄족 고체 용액의 적용의 열쇠는 바로 내구성이 좋은 결정체의 상이다.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <

# Creating labels for the answers
In the following cells, We have to create necessary labels for the answers. This is required so that we can extract answers accordingly to the questions.

In [ ]:
answer_encoding = tokenizer(
    sample_question['answer_text'],
    max_length=32,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)
tokenizer.decode(answer_encoding['input_ids'].squeeze())
labels = answer_encoding["input_ids"]
labels
labels[labels == 0] = -100
labels 

tensor([[ 4928, 42481,     1,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100]])

# Create dataset
In the following cells, we have created the dataset for input in the model. This required setting the lengths, padding and tokenizer. The data has been taken for a BioQA dataset which is specifically for this task. 

In [ ]:
class BioQADataset(Dataset):
  def __init__(
      self,
      data:pd.DataFrame,
      tokenizer:T5Tokenizer,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,
      ):
    self.data =  data
    self.tokenizer =  tokenizer
    self.source_max_token_len =  source_max_token_len
    self.target_max_token_len =  target_max_token_len
  def __len__(self):
    return len(self.data)
  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]
    source_encoding = tokenizer(
      data_row['question'],
      data_row['context'],
      max_length=self.source_max_token_len,
      padding='max_length',
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    target_encoding = tokenizer(
      data_row['answer_text'],
      max_length=self.target_max_token_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    labels = target_encoding['input_ids']
    labels[labels==0] = -100
    return dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text=data_row['answer_text'],
        input_ids=source_encoding["input_ids"].flatten(),
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )
sample_dataset = BioQADataset(df, tokenizer)
for data in sample_dataset:
  print("Question: ", data['question'])
  print("Answer text: ", data['answer_text'])
  print("Input_ids: ", data['input_ids'][:10])
  print("Labels: ", data['labels'][:10])
  break 

Question:  바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
Answer text:  교향곡
Input_ids:  tensor([  396,   775, 21778, 63122,     6,  1649,  8643, 15171, 12022,  6937])
Labels:  tensor([58337,     1,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100])


# Split Dataset into Train and Test
In the following cells, we have split the data into two parts. Test size is small due to the heavy model needed large data for training.

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.05)
train_df.shape,  val_df.shape 

((96849, 5), (5098, 5))

# Create Pytorch Lightning Module
In the following cells, we have created the lightning module. This is made when we have complex models. Hence Pytorch has introduced this to smoothen the process.

In [ ]:
 class BioDataModule(pl.LightningDataModule):
   def __init__(
       self,
       train_df: pd.DataFrame,
       test_df: pd.DataFrame,
       tokenizer:T5Tokenizer,
       batch_size: int = 8,
       source_max_token_len: int = 396,
       target_max_token_len: int = 32,
       ):
     super().__init__()
     self.train_df = train_df
     self.test_df = test_df
     self.tokenizer = tokenizer
     self.batch_size = batch_size
     self.source_max_token_len = source_max_token_len
     self.target_max_token_len = target_max_token_len
   def setup(self):
     self.train_dataset = BioQADataset(
         self.train_df,
         self.tokenizer,
         self.source_max_token_len,
         self.target_max_token_len
         )
     self.test_dataset = BioQADataset(
     self.test_df,
     self.tokenizer,
     self.source_max_token_len,
     self.target_max_token_len
     )
   def train_dataloader(self):
     return DataLoader(
         self.train_dataset,
         batch_size=self.batch_size,
         shuffle=True,
         num_workers=4
         )
   def val_dataloader(self):
     return DataLoader(
         self.test_dataset,
         batch_size=self.batch_size,
         num_workers=4
         )
   def test_dataloader(self):
     return DataLoader(
         self.test_dataset,
         batch_size=1,
         num_workers=4
         )
 BATCH_SIZE = 4
 N_EPOCHS = 6
 data_module = BioDataModule(train_df, val_df, tokenizer, batch_size=BATCH_SIZE)
 data_module.setup() 

# Building the PyTorch lightning module using T5ForConditionalGeneration model
In the following cells, we have leveraged a T5 Conditional Generator which will produce text based on some condition (this is going to be our application task – QA).

In [ ]:
class BioQAModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids, 
        attention_mask=attention_mask,
        labels=labels)
    return output.loss, output.logits
  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions":outputs, "labels": labels}
  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss
  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss
  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=0.0001)
    return optimizer
model = BioQAModel()

# Using Trainer from PyTorch-Lightning to Finetune Model Using our Dataset
We have used a trainer module to test the model and fine-tune it in the following cells. This is helpful when the model is huge, and you want to get a warmup of the model without changing the learning rate. 

In [ ]:
model = BioQAModel()
# To record the best performing model using checkpoint
checkpoint_callback = ModelCheckpoint(
    dirpath="/content/drive/MyDrive/final_pjt/ET5/model",
    filename="best-checkpoint-test",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

trainer = pl.Trainer(
    checkpoint_callback=checkpoint_callback,
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate = 30
) 

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f79085cb390>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f79085cb390>)`.
  # remove all callbacks with a type that occurs in model callbacks
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  refresh_rate = 1 if r

# Loading Tensorboard
In the following cells, we have used a Tensorboard for seeing how the model is progressing against time and epochs. This is very helpful when running huge models. 

In [ ]:
trainer.fit(model, data_module)
trainer.test()

rm: unrecognized option '--rf'
Try 'rm --help' for more information.


ModuleNotFoundError: ignored

# Predictions

In [ ]:
trained_model = BioQAModel.load_from_checkpoint("/content/drive/MyDrive/final_pjt/ET5/model/best-checkpoint-large-v1.ckpt")
# trained_model = BioQAModel.load_from_checkpoint('/content/drive/MyDrive/final_pjt/ET5/model/best-checkpoint.ckpt')
trained_model.freeze() # 

# Generate Answers for the Questions in the Validation Set
In the following cells, we have tried and tested some questions whose answers were saved with us in the valid dataset against the predicted values given by the model. As you’ll notice the T5 model is so powerful that it will predict exact values.

In [ ]:
def generate_answer(question):
  source_encoding=tokenizer(
      question['question'],
      question['context'],
      max_length = 396,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )
  generated_ids = trained_model.model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams=1,  # greedy search
      max_length=80,
      repetition_penalty=2.5,
      early_stopping=True,
      use_cache=True)
  preds = [
          tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
          for generated_id in generated_ids
  ]
  return "".join(preds)

In [ ]:
# sample_question = val_df.iloc[20]
# sample_question["question"]
# sample_question["answer_text"]  # Label Answer
# generate_answer(sample_question)  # Predicted answer
sample_question = val_df.iloc[16]
# sample_question["answer_text"]
generate_answer(sample_question)

'1만장'

In [ ]:
 !git clone https://github.com/monologg/KoBERT-Transformers.git

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/')

In [ ]:
import tokenization_kobert

In [ ]:
from tokenization_kobert import KoBertTokenizer

In [ ]:
tokenizer_t = KoBertTokenizer.from_pretrained('monologg/kobert')

In [ ]:
!pip install tensorflow #텐서플로우 설치
!pip install sklearn #사이킷런 설치
!pip install konlpy

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
okt = Okt()
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
manhattan_d = manhattan_distances(tfidf_normalized[0:1],tfidf_normalized[1:2])

In [ ]:
def cosin_sim(t_list, question):
  t_list['score']=0.0
  t_list.reset_index(drop=True, inplace=True)
  for i in range(len(t_list)):
      sentences = (t_list['context'][i], question)
      tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
      cos_similar = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
      t_list['score'][i] = cos_similar[0][0]
  # return cos_similar[0][0], type(cos_similar[0][0])
  return t_list.loc[t_list['score'].idxmax()]['context']

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
def in_title(question, dat):
  dat = dat.drop_duplicates(['context'], ignore_index = True)
  t_list = []
  con_list = []
  for i in range(len(dat)):
    if dat['title'][i].split('(')[0] in question.replace(' ', ''):  
      contet = dat.loc[i,:]
      t_list.append(contet)

    else:
      pass
  t_list = pd.DataFrame(t_list, columns = dat.columns)
  con_list = dat[dat['context'].str.contains(okt.nouns(question)[0])]
  use = pd.concat([t_list, con_list], ignore_index=True)
  if len(use) >0:
    return use
  else :
    return print('데이터 없음')


In [ ]:
def title_re(t_list, question):     

  t_list['score']=0.0
  t_list.reset_index(drop=True, inplace=True)
  for i in range(len(t_list)):
      sentences = (t_list['question'][i], question)
      tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
      tfidf_normalized = tfidf_matrix/np.sum(tfidf_matrix)
      manhattan_d = manhattan_distances(tfidf_normalized[0:1],tfidf_normalized[1:2])
      t_list['score'][i] = manhattan_d[0][0]
  t_list = t_list[t_list['score'] == min(t_list['score'])]
  return t_list


In [ ]:
def anw_test(context, question):

  source_encoding=tokenizer(
    question,
    context,
    max_length = 396,
    padding="max_length",
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)
  generated_ids = trained_model.model.generate(
    input_ids=source_encoding["input_ids"],
    attention_mask=source_encoding["attention_mask"],
    num_beams=1,  # greedy search
    max_length=80,
    repetition_penalty=2.5,
    early_stopping=True,
    use_cache=True)
  preds = [
        tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids]
  return "".join(preds), context 

In [ ]:
def last(question, dat):
  dat = dat.drop_duplicates(['context'], ignore_index = True)
  a = in_title(question,dat)   ## 타이틀, 컨텍스트 
  b= title_re(a, question)    ## 질문,타이틀 유사도 
  context =cosin_sim(b, question)    
  answer = anw_test(context, question)   ## 정답 예측, 컨텍스트 
  return answer

### 예측 정답과 실제 정답 비교용 데이터프레임 생성 


In [ ]:
sample_df = dat.sample(n = 100 , random_state=15)
re_sample = sample_df.reset_index()
ans_list = []
context_list = []

for i in range(100): 
  try:
    answer = last(re_sample.loc[i,'question'], re_sample)
    ans_list.append(answer)
    # for i in answer: 
    #   ans_list.append(a)
    #   i+=1
    #   context_list.append(a)
  except:
    ans_list.append('NaN')
    context_list.append('NaN')

In [ ]:
new_df = re_sample.copy()
for i in ans_list: 
  new_df['pred_ans'] = ans_list

new_df.drop(['answer_start','answer_end'], axis=1)

In [ ]:
@misc{ke_t5,
    author       = {KETI AIRC},
    title        = {KE-T5: Korean English T5},
    month        = may,
    year         = 2022,
    url          = {https://github.com/AIRC-KETI/ke-t5}
}